# Regression to see if salaries have gone up

In [1]:
import pandas as pd
import numpy as np

In [85]:
salary_pdf = pd.read_csv(r'C:\Users\map22\Google Drive\Old stuff\Analysis\swe_salary\swe_salary_data.tsv', sep='\t',
                        header =None, names = ['description', 'level_details', 'yoe', 'tc_details', 'tc', 'date', 'loc'])


salary_pdf['company'] = salary_pdf['description'].str.extract('^(\w+)')
salary_pdf['company'] = salary_pdf['company'].str.capitalize()

In [86]:
salary_pdf['level_details'] =salary_pdf['level_details'].str.replace('SDE II', 'SDEII')
salary_pdf['level_details'] =salary_pdf['level_details'].str.replace('SWE II', 'SWEII')
salary_pdf['level_details'] =salary_pdf['level_details'].str.replace('Software Engineer II', 'SWEII')
salary_pdf['level_details'] =salary_pdf['level_details'].str.replace('ICT ', 'ICT')
salary_pdf['level'] = salary_pdf['level_details'].str.extract('^(\w+)')
salary_pdf['level'] = salary_pdf['level'].str.upper()
def calc_level(row):
    common_dict= {'L5':'senior', 'SENIOR':'senior',
                   'L4':'swe2', 'T4':'swe2', 'L3':'swe2',
                  '4':'swe2', '5':'senior', '3':'swe2', 'T3':'swe2',
                  'E4':'swe2', 'E5':'senior',
                   'SWEIII': 'senior', 'SDEIII':'senior', 'SDEII':'swe2', 
                 'IC4':'swe2',  'IC5':'senior', 'IC3':'swe2',
                 'E6':'staff', 'L6':'staff', 'STAFF':'staff','6':'staff',
                 'T6':'staff', 'IC6':'staff'}
    level_map = {'Amazon':common_dict.copy(),
                 'Google':common_dict.copy(),
                 'Facebook': common_dict.copy(),
                 'Uber':{'5A':'senior', 'SENIOR':'senior', 'L5':'senior','L4':'swe2',
                         'SWEII':'swe2', '4':'swe2', 'L5A':'senior'},
                 'Apple':common_dict.copy(),
                 'Microsoft':{},
                 'Salesforce':common_dict.copy()}
    level_map['Amazon'].update({'L6':'senior', '6':'senior',
                                'L5':'swe2', '5':'swe2'})
    level_map['Apple'].update({'ICT4':'senior', 'ITC4':'senior',
                               'ITC3':'swe2', 'ICT3':'swe2', })
    company_map = level_map[row['company']]
    return company_map[row['level']]
salary_pdf['common_level'] = salary_pdf.apply(calc_level, axis = 1)

In [87]:
salary_pdf['year'] = pd.to_datetime(salary_pdf['date']).dt.year
salary_pdf['month'] = pd.to_datetime(salary_pdf['date']).dt.month

In [88]:
salary_pdf['west_coast'] = salary_pdf['loc'].str.contains('CA|WA')
salary_pdf['cali'] = salary_pdf['loc'].str.contains('CA')
salary_pdf['wa'] = salary_pdf['loc'].str.contains('WA')

### Regression

In [89]:
import statsmodels.formula.api as smf

In [100]:
fit_pdf = salary_pdf.loc[(salary_pdf['year'] > 2017) & salary_pdf['west_coast']]
salary_model = smf.ols('tc ~ common_level + company+ C(year) + cali -1', data = fit_pdf)
salary_results = salary_model.fit()

In [99]:
salary_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     tc   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     291.0
Date:                Tue, 21 Apr 2020   Prob (F-statistic):               0.00
Time:                        14:50:08   Log-Likelihood:                -9275.1
No. Observations:                1644   AIC:                         1.857e+04
Df Residuals:                    1633   BIC:                         1.863e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
common_level[senior]    313.1069      5.698     54.950      0.000     301.931     324.283
common_level[staff]     486.4201      8.873     54.821      0.000     469.017     503.824
common_level[swe2]      206.8222      7.088     29.181      0.000     192.920     220.724
company[T.Apple]         -6.7357      6.896     -0.977      0.329     -20.262       6.790
company[T.Facebook]      45.8590      6.311      7.266      0.000      33.480      58.238
company[T.Google]        36.9601      7.672      4.817      0.000      21.911      52.009
company[T.Salesforce]   -94.1614      9.540     -9.870      0.000    -112.874     -75.449
company[T.Uber]          48.2188      7.944      6.070      0.000      32.638      63.800
C(year)[T.2019]           1.0002      4.261      0.235      0.814      -7.357       9.358
C(year)[T.2020]          12.8935      5.001      2.578      0.010       3.085      22.703
cali[T.True]             11.3800      4.326      2.631      0.009       2.896      19.864
==============================================================================
Omnibus:                      387.819   Durbin-Watson:                   1.371
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2433.667
Skew:                           0.949   Prob(JB):                         0.00
Kurtosis:                       8.650   Cond. No.                         12.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""